In [3]:
function gradient_method(f, ∇f, x0; tol = 1e-5, max_iter = 1000, max_time = 60)
    exit_flag = 0
    
    x = copy(x0) # Cópia de x0
    iter = 0
    start_time = time()
    elapsed_time = 0.0
    fx = f(x)
    ∇fx = ∇f(x)
    while norm(∇fx) > tol
        d = -∇fx
        ∇fx_dot_d = dot(∇fx, d)
        t = 1.0
        while f(x + t*d) > fx + 0.5*t*∇fx_dot_d
            t = t*0.9
        end
        x = x + t*d
        fx = f(x)
        ∇fx = ∇f(x)
        iter = iter + 1
        if iter >= max_iter
            exit_flag = 1
            break
        end
        elapsed_time = time() - start_time
        if elapsed_time >= max_time
            exit_flag = 2
            break
        end
    end
    return x, fx, ∇fx, exit_flag, iter, elapsed_time # Precisamos retornar o ponto encontrado
end

gradient_method (generic function with 1 method)

In [1]:
function newton_method(f, ∇f, H, x0; tol = 1e-5, max_iter = 1000, max_time = 60)
    exit_flag = 0
    
    x = copy(x0) # Cópia de x0
    iter = 0
    start_time = time()
    elapsed_time = 0.0
    fx = f(x)
    ∇fx = ∇f(x)
    B = H(x)
    while norm(∇fx) > tol
        d = -B\∇fx
        ∇fx_dot_d = dot(∇fx, d)
        t = 1.0
        while f(x + t*d) > fx + 0.5*t*∇fx_dot_d
            t = t*0.9
        end
        x = x + t*d
        fx = f(x)
        ∇fx = ∇f(x)
        B = H(x)
        iter = iter + 1
        if iter >= max_iter
            exit_flag = 1
            break
        end
        elapsed_time = time() - start_time
        if elapsed_time >= max_time
            exit_flag = 2
            break
        end
    end
    return x, fx, ∇fx, exit_flag, iter, elapsed_time # Precisamos retornar o ponto encontrado
end

newton_method (generic function with 1 method)

In [2]:
using Plots
gr()

Plots.GRBackend()

In [7]:
n = 1000; (Q,R) = qr(rand(n,n));
G = Q*diagm(linspace(1,100,n))*Q'; # Uma matriz nxn com número de condição 100.
g = rand(n);
f(x) = 0.5*dot(x,G*x) + dot(g,x);
∇f(x) = G*x + g;
x, fx, ∇fx, exit_flag, iter, el_time = gradient_method(f, ∇f, zeros(n));
x, fx, ∇fx, exit_flag, iter, el_time = gradient_method(f, ∇f, zeros(n)); # Rodando duas vezes
println("exit_flag = $exit_flag")
println("iter = $iter")
println("el_time = $el_time")

H(x) = G
x, fx, ∇fx, exit_flag, iter, el_time = newton_method(f, ∇f, H, zeros(n));
x, fx, ∇fx, exit_flag, iter, el_time = newton_method(f, ∇f, H, zeros(n)); # Rodando duas vezes
println("exit_flag = $exit_flag")
println("iter = $iter")
println("el_time = $el_time")

exit_flag = 0
iter = 200
el_time = 1.1184461116790771
exit_flag = 0
iter = 1
el_time = 0.012897014617919922


In [9]:
f(x) = x[1]^4 + x[2]^4
∇f(x) = [4*x[1]^3; 4*x[2]^3]
H(x) = [12*x[1]^2   0.0; 0.0    12*x[2]^2]
x0 = [1.0; 2.0]

x, fx, ∇fx, exit_flag, iter, el_time = gradient_method(f, ∇f, x0);
x, fx, ∇fx, exit_flag, iter, el_time = gradient_method(f, ∇f, x0);
println("exit_flag = $exit_flag")
println("iter = $iter")
println("el_time = $el_time")

x, fx, ∇fx, exit_flag, iter, el_time = newton_method(f, ∇f, H, x0);
x, fx, ∇fx, exit_flag, iter, el_time = newton_method(f, ∇f, H, x0); # Rodando duas vezes
println("exit_flag = $exit_flag")
println("iter = $iter")
println("el_time = $el_time")

exit_flag = 0
iter = 852
el_time = 0.0010190010070800781
exit_flag = 0
iter = 13
el_time = 2.09808349609375e-5


In [10]:
#Pkg.add("ForwardDiff")
using ForwardDiff

In [12]:
f(x) = (x[1] - 1)^2 + 100*(x[2] - x[1]^2)^2 # Rosenbrock
x0 = [-1.2; 1.1]

2-element Array{Float64,1}:
 -1.2
  1.1

In [13]:
ForwardDiff.gradient(f, x0)

2-element Array{Float64,1}:
 -167.6
  -68.0

In [14]:
∇f(x) = ForwardDiff.gradient(f, x)

∇f (generic function with 1 method)

In [15]:
∇f(x0)

2-element Array{Float64,1}:
 -167.6
  -68.0

In [16]:
H(x) = ForwardDiff.hessian(f, x)

H (generic function with 1 method)

In [17]:
H(x0)

2x2 Array{Float64,2}:
 1290.0  480.0
  480.0  200.0

In [48]:
function newton_method(f, x0; tol = 1e-5, max_iter = 1000, max_time = 60)
    exit_flag = 0
    ∇f(x) = ForwardDiff.gradient(f, x)
    H(x) = ForwardDiff.hessian(f, x)
    
    x = copy(x0) # Cópia de x0
    iter = 0
    start_time = time()
    elapsed_time = 0.0
    fx = f(x)
    ∇fx = ∇f(x)
    B = H(x)
    while norm(∇fx) > tol
        d = -B\∇fx
        ∇fx_dot_d = dot(∇fx, d)
        if ∇fx_dot_d > -1e-6 * norm(∇fx) * norm(d)
            d = -∇fx
            ∇fx_dot_d = dot(∇fx, d)
        elseif norm(d) < 1e-6 * norm(∇fx)
            d = -∇fx
            ∇fx_dot_d = dot(∇fx, d)
        end
        t = 1.0
        while f(x + t*d) > fx + 0.5*t*∇fx_dot_d
            t = t*0.9
        end
        x = x + t*d
        fx = f(x)
        ∇fx = ∇f(x)
        B = H(x)
        iter = iter + 1
        if iter >= max_iter
            exit_flag = 1
            break
        end
        elapsed_time = time() - start_time
        if elapsed_time >= max_time
            exit_flag = 2
            break
        end
    end
    return x, fx, ∇fx, exit_flag, iter, elapsed_time # Precisamos retornar o ponto encontrado
end

newton_method (generic function with 2 methods)

In [19]:
function gradient_method(f, x0; tol = 1e-5, max_iter = 1000, max_time = 60)
    exit_flag = 0
    ∇f(x) = ForwardDiff.gradient(f, x)
    
    x = copy(x0) # Cópia de x0
    iter = 0
    start_time = time()
    elapsed_time = 0.0
    fx = f(x)
    ∇fx = ∇f(x)
    while norm(∇fx) > tol
        d = -∇fx
        ∇fx_dot_d = dot(∇fx, d)
        t = 1.0
        while f(x + t*d) > fx + 0.5*t*∇fx_dot_d
            t = t*0.9
        end
        x = x + t*d
        fx = f(x)
        ∇fx = ∇f(x)
        iter = iter + 1
        if iter >= max_iter
            exit_flag = 1
            break
        end
        elapsed_time = time() - start_time
        if elapsed_time >= max_time
            exit_flag = 2
            break
        end
    end
    return x, fx, ∇fx, exit_flag, iter, elapsed_time # Precisamos retornar o ponto encontrado
end

gradient_method (generic function with 2 methods)

In [49]:
f(x) = (x[1] - 1)^2 + 100*(x[2] - x[1]^2)^2 # Rosenbrock
x0 = [-1.2; 1.1]

x, fx, ∇fx, exit_flag, iter, el_time = gradient_method(f, x0);
x, fx, ∇fx, exit_flag, iter, el_time = gradient_method(f, x0);
println("exit_flag = $exit_flag")
println("iter = $iter")
println("el_time = $el_time")

x, fx, ∇fx, exit_flag, iter, el_time = newton_method(f, x0);
x, fx, ∇fx, exit_flag, iter, el_time = newton_method(f, x0); # Rodando duas vezes
println("exit_flag = $exit_flag")
println("iter = $iter")
println("el_time = $el_time")
x

exit_flag = 1
iter = 1000
el_time = 0.010305166244506836
exit_flag = 0
iter = 21
el_time = 0.0001590251922607422


2-element Array{Float64,1}:
 1.0
 1.0

In [23]:
function eh_def_pos(A)
    return all(eig(A)[1] .> 0)
end

eh_def_pos (generic function with 1 method)

In [53]:
x0 = rand(2)
eh_def_pos(H(x0))

false

In [27]:
x0

2-element Array{Float64,1}:
 0.0960576
 0.677739 

In [54]:
f(x) = (x[1] - 1)^2 + 100*(x[2] - x[1]^2)^2 # Rosenbrock

x, fx, ∇fx, exit_flag, iter, el_time = gradient_method(f, x0);
x, fx, ∇fx, exit_flag, iter, el_time = gradient_method(f, x0);
println("exit_flag = $exit_flag")
println("iter = $iter")
println("el_time = $el_time")

x, fx, ∇fx, exit_flag, iter, el_time = newton_method(f, x0);
x, fx, ∇fx, exit_flag, iter, el_time = newton_method(f, x0); # Rodando duas vezes
println("exit_flag = $exit_flag")
println("iter = $iter")
println("el_time = $el_time")
x

exit_flag = 1
iter = 1000
el_time = 0.019587039947509766
exit_flag = 0
iter = 15
el_time = 0.00015807151794433594


2-element Array{Float64,1}:
 1.0
 1.0